In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import path
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta
import time
import warnings
import datetime
import math
from scipy.ndimage.interpolation import shift
import shapely.wkt
from scipy.stats import pearsonr,spearmanr,kendalltau, linregress
from scipy.optimize import curve_fit 

warnings.filterwarnings('ignore')
import seaborn as sns
sns.set(font_scale=1.4) 
from my_functions import sat_vap_press, vap_press, hot_dry_windy, haines
from os.path import exists

from sklearn.preprocessing import OneHotEncoder


# the goal of this file should be to preprocess the whole yearly database into the form with different resource types as columns

In [4]:
types = {'crews': ['Crew, Camp', 'Crew, Type 1', 'Crew, Type 2', 'Crew, Type 2IA'],
        'engines': ['Engine, Type 1','Engine, Type 2','Engine, Type 3',
                     'Engine, Type 4','Engine, Type 5','Engine, Type 6','Engine, Type 7'],
        'air': ['Air Attack', 'Airtanker, Type 1', 'Airtanker, Type 2', 'Airtanker, Type 3', 'Airtanker, VLAT', 'Fixed Wing, ASM', 
               'Fixed Wing, Air Tactical', 'Fixed Wing, LEAD', 'Fixed Wing, Recon', 'Helicopter, Type 1', 'Helicopter, Type 2', 
               'Helicopter, Type 3','Single Engine Airtanker'],
        'construction': ['Tractor Plow, Type 1','Tractor Plow, Type 2','Tractor Plow, Type 3','Tractor Plow, Type 4',
                         'Tractor Plow, Type 5','Tractor Plow, Type 6', 'Dozer', 'Masticator'],
        'overhead':['Overhead']}

resources_file='../YYYY_RESOURCES.xlsx'

years = [2019,2020,2021]
#years=[2021]

for jj in range(len(years)):
    
   
    resources_all = pd.read_excel(resources_file.replace('YYYY', str(years[jj]))).iloc[:,0:9]
    resources_all.columns = ['INC209R_IDENTIFIER','REPORT_FROM_DATE','REPORT_TO_DATE','PCT_CONTAINED_COMPLETED',
                             'INCIDENT_NAME', 'IRWIN_IDENTIFIER','CODE_NAME','RESOURCE_QUANTITY','RESOURCE_PERSONNEL']
    
    #print(resources_all)
    
    ohe = OneHotEncoder()
    transformed = ohe.fit_transform(resources_all[['CODE_NAME']])
    resources_all[ohe.categories_[0]] = transformed.toarray()
    print(resources_all.columns.values)
    
    #make the data frame of the stuff we want to save, we will add to it
    resources_save = resources_all[['INC209R_IDENTIFIER','REPORT_FROM_DATE','REPORT_TO_DATE','PCT_CONTAINED_COMPLETED',
                             'INCIDENT_NAME', 'IRWIN_IDENTIFIER','CODE_NAME','RESOURCE_QUANTITY','RESOURCE_PERSONNEL']]
    
    #do the multiplication of the resource and personnal numbers, before we add by subtype
    resources_quantity = resources_all[types['crews']+types['engines']+types['air']+types['construction']+types['overhead']].multiply(resources_all['RESOURCE_QUANTITY'], axis=0)
    resources_personnel = resources_all[types['crews']+types['engines']+types['air']+types['construction']+types['overhead']].multiply(resources_all['RESOURCE_PERSONNEL'], axis=0)

    
    #crews
    resources_save['crew_quantity'] = resources_quantity[types['crews']].sum(axis=1)
    resources_save['crew_personnel'] = resources_personnel[types['crews']].sum(axis=1)

    #engines 
    resources_save['engine_quantity'] = resources_quantity[types['engines']].sum(axis=1)
    resources_save['engine_personnel'] = resources_personnel[types['engines']].sum(axis=1)
    
    #air
    resources_save['air_quantity'] = resources_quantity[types['air']].sum(axis=1)
    resources_save['air_personnel'] = resources_personnel[types['air']].sum(axis=1)

    #construction 
    resources_save['construction_quantity'] = resources_quantity[types['construction']].sum(axis=1)
    resources_save['construction_personnel'] = resources_personnel[types['construction']].sum(axis=1)
    
    #overhead
    resources_save['overhead_personnel'] = resources_personnel[types['overhead']].sum(axis=1)



    print(resources_save)
    
    resources_save.to_excel('../'+str(years[jj])+'+_PROCESSED_RESOURCES.xlsx') #daily averages


['INC209R_IDENTIFIER' 'REPORT_FROM_DATE' 'REPORT_TO_DATE'
 'PCT_CONTAINED_COMPLETED' 'INCIDENT_NAME' 'IRWIN_IDENTIFIER' 'CODE_NAME'
 'RESOURCE_QUANTITY' 'RESOURCE_PERSONNEL' 'ALS Ambulance' 'Air Attack'
 'Airtanker, Type 1' 'Airtanker, Type 2' 'Airtanker, Type 3'
 'Airtanker, VLAT' 'Boat, Transportation' 'Crash Rescue (Aircraft)'
 'Crew, Camp' 'Crew, Type 1' 'Crew, Type 2' 'Crew, Type 2IA' 'Dozer'
 'Engine, Type 1' 'Engine, Type 2' 'Engine, Type 3' 'Engine, Type 4'
 'Engine, Type 5' 'Engine, Type 6' 'Engine, Type 7' 'Fixed Wing, ASM'
 'Fixed Wing, Air Tactical' 'Fixed Wing, LEAD' 'Fixed Wing, Recon'
 'Food Dispensing Unit' 'Hazardous Materials Unit' 'Heavy Rescue Unit'
 'Helicopter, Type 1' 'Helicopter, Type 2' 'Helicopter, Type 3'
 'Illumination Unit (Lighting)' 'Masticator' 'Medic Squad'
 'Mobile Communications Center' 'Mobile Kitchen Unit' 'Overhead'
 'Rescue Unit' 'Rescue/Medical Unit' 'Single Engine Airtanker' 'Skidgine'
 'Technical Rescue Team' 'Tender, Potable Water' 'Tender, Wa

['INC209R_IDENTIFIER' 'REPORT_FROM_DATE' 'REPORT_TO_DATE'
 'PCT_CONTAINED_COMPLETED' 'INCIDENT_NAME' 'IRWIN_IDENTIFIER' 'CODE_NAME'
 'RESOURCE_QUANTITY' 'RESOURCE_PERSONNEL' 'ALS Ambulance' 'ASM'
 'Aerial Apparatus' 'Air Attack' 'Air Attack Platform' 'Airtanker, Type 1'
 'Airtanker, Type 2' 'Airtanker, Type 3' 'Airtanker, VLAT' 'Ambulance'
 'Boat' 'Boat, Fire' 'Boat, Rescue' 'Boat, Transportation'
 'Crash Rescue (Aircraft)' 'Crew, Camp' 'Crew, Type 1' 'Crew, Type 2'
 'Crew, Type 2IA' 'Dozer' 'Engine, Type 1' 'Engine, Type 2'
 'Engine, Type 3' 'Engine, Type 4' 'Engine, Type 5' 'Engine, Type 6'
 'Engine, Type 7' 'Excavator' 'Feller Buncher' 'Fixed Wing, ASM'
 'Fixed Wing, Air Tactical' 'Fixed Wing, LEAD' 'Fixed Wing, Recon'
 'Food Dispensing Unit' 'GIS Unit' 'Heavy (Type 1)' 'Helicopter, Type 1'
 'Helicopter, Type 2' 'Helicopter, Type 3' 'Laundry, Mobile' 'Lead Plane'
 'Masticator' 'Medic Squad' 'Mobile Communications Center'
 'Mobile Kitchen Unit' 'Overhead' 'Pumper Cat' 'Reconnaissance

# Development code

In [10]:
#print(resources_all.groupby(by=['REPORT_FROM_DATE', 'REPORT_TO_DATE','CODE_NAME']).sum())
#print(resources_all.pivot(index=['REPORT_FROM_DATE', 'REPORT_TO_DATE','IRWIN_IDENTIFIER'], columns='CODE_NAME'))

In [29]:
resources_sub = resources_all[['','CODE_NAME', 'RESOURCE_QUANTITY', 'RESOURCE_PERSONNEL']]
resources_sub

,CODE_NAME,RESOURCE_QUANTITY,RESOURCE_PERSONNEL
0,Dozer,1.0,1.0
1,Overhead,NaN,0.0
2,"Engine, Type 6",1.0,1.0
3,Dozer,1.0,1.0
4,Overhead,NaN,0.0
...,...,...,...
167743,Dozer,1.0,1.0
167744,Overhead,NaN,0.0
167745,"Engine, Type 6",13.0,21.0
167746,Dozer,3.0,3.0


In [39]:
print(types['engines']+['RESOURCE_QUANTITY', 'RESOURCE_PERSONNEL'])


['Engine, Type 1', 'Engine, Type 2', 'Engine, Type 3', 'Engine, Type 4', 'Engine, Type 5', 'Engine, Type 6', 'Engine, Type 7', 'RESOURCE_QUANTITY', 'RESOURCE_PERSONNEL']


In [57]:
resources_quantity = resources_all[types['crews']+types['engines']+types['air']+types['construction']].multiply(resources_all['RESOURCE_QUANTITY'], axis=0)
resources_personnel = resources_all[types['crews']+types['engines']+types['air']+types['construction']].multiply(resources_all['RESOURCE_PERSONNEL'], axis=0)
print(resources_personnel[0:100])

    Crew, Camp  Crew, Type 1  Crew, Type 2  Crew, Type 2IA  Engine, Type 1  \
0          0.0           0.0           0.0             0.0             0.0   
1          0.0           0.0           0.0             0.0             0.0   
2          0.0           0.0           0.0             0.0             0.0   
3          0.0           0.0           0.0             0.0             0.0   
4          0.0           0.0           0.0             0.0             0.0   
..         ...           ...           ...             ...             ...   
95         0.0           0.0           0.0             0.0             0.0   
96         0.0           0.0           0.0             0.0             0.0   
97         0.0           0.0          20.0             0.0             0.0   
98         0.0           0.0           0.0             0.0             0.0   
99        10.0           0.0           0.0             0.0             0.0   

    Engine, Type 2  Engine, Type 3  Engine, Type 4  Engine, Typ

In [55]:
print(resources_quantity[types['crews']].sum(axis=1)[0:100])
print(resources_personnel[types['crews']].sum(axis=1)[0:100])

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
     ... 
95    0.0
96    0.0
97    1.0
98    0.0
99    1.0
Length: 100, dtype: float64
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
95     0.0
96     0.0
97    20.0
98     0.0
99    10.0
Length: 100, dtype: float64


In [44]:
resources_sub = resources_all[types['engines']+['RESOURCE_QUANTITY', 'RESOURCE_PERSONNEL']]
print(resources_sub)

print(resources_all[types['engines']])
print(resources_all[types['engines']].sum(axis=1))

        Engine, Type 1  Engine, Type 2  Engine, Type 3  Engine, Type 4  \
0                  0.0             0.0             0.0             0.0   
1                  0.0             0.0             0.0             0.0   
2                  0.0             0.0             0.0             0.0   
3                  0.0             0.0             0.0             0.0   
4                  0.0             0.0             0.0             0.0   
...                ...             ...             ...             ...   
167743             0.0             0.0             0.0             0.0   
167744             0.0             0.0             0.0             0.0   
167745             0.0             0.0             0.0             0.0   
167746             0.0             0.0             0.0             0.0   
167747             0.0             0.0             0.0             0.0   

        Engine, Type 5  Engine, Type 6  Engine, Type 7  RESOURCE_QUANTITY  \
0                  0.0            

In [31]:
resources_sub = resources_all[['Overhead','Dozer','Engine, Type 6', 'RESOURCE_QUANTITY', 'RESOURCE_PERSONNEL']]

resources_sub['Number of Engine, Type 6'] = resources_sub['Engine, Type 6']*resources_sub['RESOURCE_QUANTITY']
resources_sub['Number of Engine, Type 6 Workers'] = resources_sub['Engine, Type 6']*resources_sub['RESOURCE_PERSONNEL']

resources_sub


,Overhead,Dozer,"Engine, Type 6",RESOURCE_QUANTITY,RESOURCE_PERSONNEL,"Number of Engine, Type 6","Number of Engine, Type 6 Workers"
0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
1,1.0,0.0,0.0,NaN,0.0,NaN,0.0
2,0.0,0.0,1.0,1.0,1.0,1.0,1.0
3,0.0,1.0,0.0,1.0,1.0,0.0,0.0
4,1.0,0.0,0.0,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...
167743,0.0,1.0,0.0,1.0,1.0,0.0,0.0
167744,1.0,0.0,0.0,NaN,0.0,NaN,0.0
167745,0.0,0.0,1.0,13.0,21.0,13.0,21.0
167746,0.0,1.0,0.0,3.0,3.0,0.0,0.0
